**Problema Laborator 3:**
Date:
$$
\begin{align*}
& P(C=1)=0.0005\\
& P(I=1|C=0)=0.01 & P(I=1|C=1)=0.03\\
& P(A=1|C=0,I=0)=0.0001 & P(A=1|C=1,I=0)=0.02\\
& P(A=1|C=0,I=1)=0.95 & P(A=1|C=1,I=1)=0.98
\end{align*}
$$

**Raspuns teoretic (punctul 4 - Bonus):**

2. $
P(C=1|A=1)=\dfrac{P(A=1|C=1)P(C=1)}{P(A=1|C=1)P(C=1)+P(A=1|C=0)P(C=0)}
$

Avem:
\begin{align*}
P(A=1|C=1)  & =P(A=1|C=1,I=0)P(I=0|C=1)+P(A=1|C=1,I=1)P(I=1|C=1)\\
& =0.02\cdot0.97+0.98\cdot0.03=0.0488;\\
P(A=1|C=0)  & =P(A=1|C=0,I=0)P(I=0|C=0)+P(A=1|C=0,I=1)P(I=1|C=0)\\
& =0.0001\cdot0.99+0.95\cdot0.01=0.009599.
\end{align*}
Deci
$$
\begin{align*}
P(C=1|A=1)&=\frac{0.0488\cdot0.0005}{0.0488\cdot0.0005+0.009599\cdot0.9995}\\
&=0.0025367516\cong 0.25\%.
\end{align*}
$$

2.
$
P(I=1|A=0) =\dfrac{P(A=0|I=1)P(I=1)}{P(A=0|I=1)P(I=1)+P(A=0|I=0)P(I=0)}
$

Avem:
\begin{align*}
P(A=0|I=1)P(I=1)  & =P(A=0|I=1,C=0)P(C=0|I=1)P(I=1)+P(A=0|I=1,C=1)P(C=1|I=1)P(I=1)\\
& =P(A=0|I=1,C=0)P(I=1|C=0)P(C=0)+P(A=0|I=1,C=1)P(I=1|C=1)P(C=1)\\
& =0.05\cdot0.01\cdot0.9995+0.02\cdot0.03\cdot0.0005=0.00050005;\\
P(A=0|I=0)P(I=0) & =P(A=0|I=0,C=0)P(C=0|I=0)P(I=0)+P(A=0|I=0,C=1)P(C=1|I=0)P(I=0)\\
& =P(A=0|I=0,C=0)P(I=0|C=0)P(C=0)+P(A=0|I=0,C=1)P(I=0|C=1)P(C=1)\\
& =0.9999\cdot0.99\cdot0.9995+0.98\cdot0.97\cdot0.0005=0.98988.
\end{align*}
Deci
$
P(I=1|A=0) =\dfrac{0.00050005}{0.00050005+0.98988}=0.00050491\cong 0.05\%.
$

Cerinta poate fi inteleasa si ca $P(I=1,A=0).$ In cazul acesta, rezultatul e numaratorul fractiei de mai sus, adica 0.00050005.

In [5]:
import pandas as pd
import pymc3 as pm

**Punctul 1.**

In [43]:
sample = []
for i in range(2):
    model = pm.Model()

    with model:
        cutremur = pm.Bernoulli('C', p=0.0005)
        incendiu_p = pm.math.switch(cutremur, 0.03, 0.01)
        incendiu = pm.Bernoulli('I', p=incendiu_p)
        alarma_p = pm.math.switch(incendiu, pm.math.switch(cutremur, 0.98, 0.95), pm.math.switch(cutremur, 0.02, 0.0001))
        alarma = pm.Bernoulli('A', p=alarma_p, observed=i)          # i = 0 sau 1, coresp. pct. 3, resp. 2
        trace = pm.sample(50000)                                    # nr. iteratii mai mare --> precizie mai mare
    sample.append(trace)

c:\Users\adria\AppData\Local\Programs\Python\Python310\lib\site-packages\deprecat\classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
BinaryGibbsMetropolis: [C, I]


Sampling 4 chains for 1_000 tune and 50_000 draw iterations (4_000 + 200_000 draws total) took 67 seconds.
c:\Users\adria\AppData\Local\Programs\Python\Python310\lib\site-packages\deprecat\classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
BinaryGibbsMetropolis: [C, I]


Sampling 4 chains for 1_000 tune and 50_000 draw iterations (4_000 + 200_000 draws total) took 66 seconds.


**Punctele 2 si 3**

In [46]:
dictionary = {
              'cutremur': sample[1]['C'].tolist(),  #pt. pct. 2
              'incendiu': sample[0]['I'].tolist()   #pt. pct. 3
              }
df = pd.DataFrame(dictionary)

In [47]:
p_cutremur = df[(df['cutremur'] == 1)].shape[0] / df.shape[0]
p_incendiu = df[(df['incendiu'] == 1)].shape[0] / df.shape[0]
print('P(C=1|A=1)=',p_cutremur)
print('P(I=1|A=0)=',p_incendiu)

P(C=1|A=1)= 0.002595
P(I=1|A=0)= 0.0005
